In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from grabscreen import grab_screen
import cv2
import urllib.request

# This is needed to display the images.
get_ipython().magic('matplotlib inline')

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

C:\Users\motoko\AppData\Local\Temp\ipykernel_12604\382177206.py:18: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib inline')


In [3]:
# # Model preparation 
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS

'data\\mscoco_label_map.pbtxt'

In [5]:
NUM_CLASSES = 90 #need to be checked

# ## Download Model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

KeyboardInterrupt: 

as_default

Returns a context manager that makes this Graph the default graph.
Use this method with the with keyword to specify that ops created within the scope of a block should be added to this graph. In this case, once the scope of the with is exited, the previous default graph is set again as default. There is a stack, so it's ok to have multiple nested levels of as_default calls.

In [6]:
# from object_detection.utils import ops as utils_ops
# utils_ops.tf = tf.compat.v1
# tf.gfile = tf.io.gfile

detection_graph=tf.Graph()
with detection_graph.as_default():
    od_graph_def=tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph=fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        # tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map=label_map_util.load_labelmap(PATH_TO_LABELS)
categories=label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, 
                                                          use_display_name=True)

category_index=label_map_util.create_category_index(categories)

In [8]:
def load_image_into_numpy_array(image):
    (im_width, im_height)=image.size
    return np.array(image.getdata()).reshape((im_width, im_height),3).astype(np.uint8)

In [9]:

# Ensure you're in the right directory or specify the path directly
path_to_ckpt = 'ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb'

# Check if the model file exists
if not os.path.exists(path_to_ckpt):
    print("Model file not found")
else:
    print("Model file found, loading...")

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()

        with tf.io.gfile.GFile(path_to_ckpt, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        # Check to print the operations in the graph to verify if it's loaded
        print("Operations in the graph:")
        for op in detection_graph.get_operations():
            print(op.name)


Model file found, loading...
Operations in the graph:
Const
image_tensor
ToFloat
Preprocessor/map/Shape
Preprocessor/map/strided_slice/stack
Preprocessor/map/strided_slice/stack_1
Preprocessor/map/strided_slice/stack_2
Preprocessor/map/strided_slice
Preprocessor/map/TensorArray
Preprocessor/map/TensorArrayUnstack/Shape
Preprocessor/map/TensorArrayUnstack/strided_slice/stack
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2
Preprocessor/map/TensorArrayUnstack/strided_slice
Preprocessor/map/TensorArrayUnstack/range/start
Preprocessor/map/TensorArrayUnstack/range/delta
Preprocessor/map/TensorArrayUnstack/range
Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
Preprocessor/map/Const
Preprocessor/map/TensorArray_1
Preprocessor/map/while/Enter
Preprocessor/map/while/Enter_1
Preprocessor/map/while/Merge
Preprocessor/map/while/Merge_1
Preprocessor/map/while/Less/Enter
Preprocessor/map/while/Less
Preproc

In [25]:
IMAGE_SIZE=(12,8)
# IMAGE_SIZE=(480,270)
WIDTH=480
HEIGHT=270

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            # screen=cv2.resize(grab_screen(region=(0,45,1920,1120)), (WIDTH,HEIGHT))
            # screen = cv2.resize(grab_screen(region=(0,40,1280,745)), (WIDTH,HEIGHT))
            
            screen = cv2.resize(grab_screen(region=(0,45,800,650)), (WIDTH,HEIGHT))
            image_np=cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)

            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = 'image_tensor:0'

            # Each box represents a part of the image where a particular object was detected.
            boxes = 'detection_boxes:0'

            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = 'detection_scores:0'
            classes = 'detection_classes:0'
            num_detections = 'num_detections:0'
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            cv2.imshow('window',image_np)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break